# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [46]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [47]:
data = pd.io.stata.read_stata('us_job_market_discrimination.dta')

In [48]:
# number of callbacks for black-sounding names
black = sum(data[data.race=='b'].call)
print ('Number of callbacks for black-sounding names:', black)

# number of callbacks for white-sounding names
white = sum(data[data.race=='w'].call)
print ('Number of callbacks for white-sounding names:', white)

Number of callbacks for black-sounding names: 157.0
Number of callbacks for white-sounding names: 235.0


In [49]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [50]:
data.race.head()

0    w
1    w
2    b
3    b
4    w
Name: race, dtype: object

## 1.What test is appropriate for this problem? Does CLT apply?

By analyzing nearly 5000 cases, our goal is to test whether a person is more likely to receive a call back on job application depending on their races. 

The appropriate test is to see if there is the z-statistic because the sample number is high. 

Since there are 4870 resumes (n>30) and each sample is independent the central limit theorem applies.


## 2.What are the null and alternate hypotheses?

The null hypothesis is that the callback rate for blacks equals the callback rate for whites. 

The alterative hypothesis is that the callback rate for blacks does not equal the callback rate for whites.

## 3.Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [51]:
w = data[data.race=='w']
b = data[data.race=='b']

# Compute the callback rates for both races

# Number of applications
apply_b = len(data[data.race=='b'])
apply_w = len(data[data.race=='w'])

# Number of callbacks
call_b = sum(data[data.race=='b'].call)
call_w = sum(data[data.race=='w'].call)

rate_b = call_b / apply_b
rate_w = call_w / apply_w

print('Callback rate for black applicants:', rate_b)
print('Callback rate for white applicants:', rate_w)

# Compute the margin of error
z_critical = stats.norm.ppf(q = 0.95) # z criticals
print('z-critical value: ', z_critical)             

std = np.sqrt((rate_b*(1-rate_b)/apply_b)+(rate_w*(1-rate_w)/apply_w))# standard deviation
print('Standard deviation: ', std)  

me = z_critical * std
print('Margin of error:',me)

Callback rate for black applicants: 0.064476386037
Callback rate for white applicants: 0.0965092402464
z-critical value:  1.64485362695
Standard deviation:  0.00778337058668
Margin of error: 0.0128025053394


In [52]:
# Find 95% confidence intervals
p = rate_w - rate_b

lower = p - me
upper = p + me

print ('Confidence Interval', [lower, upper])

Confidence Interval [0.019230348870042917, 0.04483535954884825]


In [53]:
# compute the p-value of one-tail test
z_stat = p / std
p_value = 1 - stats.norm.cdf(z_stat)
print('The p-value for the test of equality of callback rates is: {:1.5f}'.format(p_value))

The p-value for the test of equality of callback rates is: 0.00002


## 4.Write a story describing the statistical significance in the context or the original problem.

The is a 95% chance that the true difference of white callback rates and black callback rates is between 0.019 and 0.045. 
OR we are 95% confident that there is a significant difference between the two races in which individuals with white-sounding names are favored and called back more often (p-value = 0.00002).
Thus, this statistic suggests that race does, in fact, have a significant impact on the rate of callbacks.


## 5.Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No, the analysis only accounts for relationship between race and callbacks. However, there are also other varialbes, such as education and years of experience that can affect the callback success. 
We must individually test the signiicance of these variables to callback rate to determine which is the best to determine callback success.